In [ ]:
!pip install -U sentence-transformers
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 724.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 2.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=088fda977c5a32859bbd7ff1c5a5d6acfa83d11a3f8baf7f51e79de0204787da
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
import nltk

# Ensure the Punkt tokenizer is downloaded
nltk.download('punkt')

def split_into_sentences(text):
    sentences = text.split('.')
    filtered_list = [string for string in sentences if string]
    return filtered_list

def split_into_strings(text):
    sentences = text.split('.')
    sentences = [string for string in sentences if string]
    strings = [s.split(',') for s in sentences]
    filtered_list = [e for e in strings if len(e) != 1 and e[0] != '']

    list = []
    for e in strings:
      for i in e:
        list.append(i)

    return list

data = [
    {
        "criteria": "Memory allocated with malloc() must be explicitly deallocated with free().",
        "answer": "Memory allocated with new must be explicitly deallocated with free(), whereas memory allocated with malloc() should be deallocated with delete.",
        "score": "False"
    },
      {
        "criteria": "Memory allocated with malloc() must be explicitly deallocated with free().",
        "answer": "Memory allocated with malloc must be explicitly deallocated with free().",
        "score": "True"
    },
    {
        "criteria": "Memory allocated with malloc() must be explicitly deallocated with free().",
        "answer": "Memory allocated with free() must be explicitly deallocated with malloc(), whereas memory allocated with malloc() should be deallocated with delete.",
        "score": "False"
    },
    {
        "criteria": "Memory allocated with malloc() must be explicitly deallocated with free().",
        "answer": "Memory allocated with malloc() must be explicitly deallocated with free(), whereas memory allocated with new should be deallocated with delete.",
        "score": "True"
    },
    {
        "criteria": "Memory allocated with malloc() must be explicitly deallocated with free(), whereas memory allocated with new should be deallocated with delete.",
        "answer": "Memory allocated with new must be explicitly deallocated with free(), whereas memory allocated with malloc() should be deallocated with delete.",
        "score": "False"
    },
    {
        "criteria": "Memory allocated with malloc() must be explicitly deallocated with free().",
        "answer": "Memory allocated with malloc() must be explicitly deallocated with delete.",
        "score": "False"
    },
    {
        "criteria": "Memory allocated with malloc() must be explicitly deallocated with free().",
        "answer": "Memory allocated with malloc() must be explicitly deallocated with free(), whereas memory allocated with new should be deallocated with delete.",
        "score": "True"
    },
    {
        "criteria": "Memory allocated with new must be explicitly deallocated with delete.",
        "answer": "Memory allocated with malloc() must be explicitly deallocated with free(), whereas memory allocated with new should be deallocated with delete.",
        "score": "True"
    },
    {
        "criteria": "new is an operator in C++",
        "answer": "Memory allocated with malloc() must be explicitly deallocated with free(), whereas memory allocated with new should be deallocated with delete.",
        "score": "False"
    },
    {
        "criteria": "new is an operator in C++",
        "answer": "Memory allocated with malloc() must be explicitly deallocated with free(), whereas memory allocated with new should be deallocated with delete. new and delete are C++ operators.",
        "score": "True"
    },
    {
        "criteria": "malloc is a function in C",
        "answer": "Memory allocated with malloc() must be explicitly deallocated with free(), whereas memory allocated with new should be deallocated with delete.",
        "score": "False"
    },
    {
        "criteria": "The Single Responsibility Principle",
        "answer": "The Single Responsibility Principle, the Open-Closed Principle, the Liskov Substitution Principle, the Interface Segregation Principle, the Dependency Inversion Principle",
        "score": "True"
    },
    {
        "criteria": "Encapsulation",
        "answer": "The Single Responsibility Principle, the Open-Closed Principle, the Liskov Substitution Principle, the Interface Segregation Principle, the Dependency Inversion Principle",
        "score": "False"
    },
    {
        "criteria": "A pointer stores the memory address of a variable.",
        "answer": "A pointer in C++ is a variable that holds the memory address of another variable. It allows you to indirectly access the value of the variable it points to. Here's an example",
        "score": "True"
    },
    {
        "criteria": "A pointer can be dereferenced with '&'",
        "answer": "A pointer in C++ is a variable that holds the memory address of another variable. It allows you to indirectly access the value of the variable it points to. Here's an example",
        "score": "False"
    }
]

problem_1 = [
    {
        "criteria": "new is an operator in C++",
        "answer": "Memory allocated with malloc() must be explicitly deallocated with free(), whereas memory allocated with new should be deallocated with delete.",
        "score": "False"
    },
    {
        "criteria": "new is an operator in C++",
        "answer": "Memory allocated with malloc() must be explicitly deallocated with free(), whereas memory allocated with new should be deallocated with delete. new and delete are C++ operators.",
        "score": "True"
    }
]

def str_to_bool(s):
    if s.lower() == 'true':
        return True
    elif s.lower() == 'false':
        return False
    else:
        raise ValueError("Cannot convert {} to a bool".format(s))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [9]:
from sentence_transformers import SentenceTransformer, util, InputExample, losses
import numpy as np
import torch.nn.functional as F
import os

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
#model = SentenceTransformer('model.zip')

In [13]:
from torch.utils.data import DataLoader
from tensorflow import keras
import tensorflow as tf
from sentence_transformers import evaluation


train_data = []

train_data.append({
    "sentences" : ["Memory allocated with malloc() must be explicitly deallocated with free()", "Memory allocated with new must be explicitly deallocated with free()"],
    "label" : 0.7
    })

train_data.append({
    "sentences" : ["Memory allocated with malloc() must be explicitly deallocated with free()", "Memory allocated with malloc must be explicitly deallocated with free()"],
    "label" : 0.95
    })

train_data.append({
    "sentences" : ["Memory allocated with malloc() must be explicitly deallocated with delete", "Memory allocated with malloc must be explicitly deallocated with free()"],
    "label" : 0.7
    })

train_data.append({
    "sentences" : ["Memory allocated with new must be explicitly deallocated with delete", "Memory allocated with malloc must be explicitly deallocated with free()"],
    "label" : 0.65
    })

train_data.append({
    "sentences" : ["Memory allocated with new must be explicitly deallocated with delete", "Memory allocated with new must be explicitly deallocated with free()"],
    "label" : 0.7
    })

train_data.append({
    "sentences" : ["Memory allocated with new must be explicitly deallocated with delete", "Memory allocated with delete must be explicitly deallocated with new"],
    "label" : 0.5
    })

train_data.append({
    "sentences" : ["new is an operator in C++", "new and delet are c++ operators"],
    "label" : 0.81
    })

train_data.append({
    "sentences" : ["A pointer stores the memory address of a variable", "A pointer in C++ is a variable that holds the memory address of another variable"],
    "label" : 0.9
    })

''' train_examples = [InputExample(texts=['My first sentence', 'My second sentence'], label=0.8),
   InputExample(texts=['Another pair', 'Unrelated sentence'], label=0.3)]
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16) '''
train_examples = []
for e in train_data:
  train_examples.append(InputExample(texts=e['sentences'], label=e['label']))

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

#Define your train examples. You need more than just two examples...
train_examples = [InputExample(texts=['My first sentence', 'My second sentence'], label=0.8),
    InputExample(texts=['Another pair', 'Unrelated sentence'], label=0.3)]

#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

checkpoint_path = "training_2_kerascb/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
''' cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=False,
                                                 verbose=1) '''

''' # Train the model with the new callback
model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training '''
sentences1 = ["new is an operator in C++", "A pointer stores the memory address of a variable"]
sentences2 = ["new and delet are c++ operators", "A pointer in C++ is a variable that holds the memory address of another variable"]

evaluator = evaluation.EmbeddingSimilarityEvaluator(sentences1, sentences2, [0.81, 0.9])


#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=5, warmup_steps=100, evaluator=evaluator, evaluation_steps=500, output_path=checkpoint_dir, save_best_model=True)


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
my_model = SentenceTransformer(checkpoint_dir)

results = []
labels = []
for e in data:
  print("===================================comparison start=======================================")
  print("original answer: " + str(e['answer']))
  print("original criteria: " + str(e['criteria']))
  evaluated = False
  label = str_to_bool(e['score'])
  labels.append(label)

  res  = False

  sentences_crit = split_into_sentences(e['criteria'])
  sentences_answer = split_into_strings(e['answer'])

  print(sentences_crit)
  print(sentences_answer)

  #Compute embedding for both lists
  crit_e = my_model.encode(sentences_crit, convert_to_tensor=True)
  print("crit_e len: " + str(len(crit_e)))
  answ_e = my_model.encode(sentences_answer, convert_to_tensor=True)
  print("ans_e len: " + str(len(answ_e)))


  #Compute cosine-similarities
  #cosine_scores = util.cos_sim(crit_e, answ_e)
  cosine_scores = util.pytorch_cos_sim(crit_e, answ_e)

  euclidean_dist_arr = []
  for crit in crit_e:
    for ans in answ_e:
      crit_arr = crit.numpy()
      answ_arr = ans.numpy()
      dist = answ_arr - crit_arr
      euclidean_dist_arr.append(np.linalg.norm(dist))

  print("cos score:")
  print(cosine_scores)
  print("eucl score:")
  print(euclidean_dist_arr)
  print(type(cosine_scores))

  for row in cosine_scores:
    for val in row:

      print(val.item())
      if(val.item() > 0.8):
        for e in euclidean_dist_arr:
          if (e < 0.51):
            res = True
            evaluated = True
            break

    if(evaluated == True):
      break

  print("label: " + str(label))
  print("outcome: " + str(res))
  if (res == label):
    results.append(True)
  else:
    results.append(False)



  print("=================================comparison end=========================================")


print("results len:")
print(len(results))
print(results)

print("==========================================================================")
print("===============================evaluation=================================")
print("==========================================================================")

correct_count = 0
for e in results:
  if (e == True):
    correct_count = correct_count + 1

print('\n')
print('\n')

score = correct_count / len(results)

print(score)



===================================comparison start=======================================
original answer: Memory allocated with new must be explicitly deallocated with free(), whereas memory allocated with malloc() should be deallocated with delete.
original criteria: Memory allocated with malloc() must be explicitly deallocated with free().
['Memory allocated with malloc() must be explicitly deallocated with free()']
['Memory allocated with new must be explicitly deallocated with free()', ' whereas memory allocated with malloc() should be deallocated with delete']
crit_e len: 1
ans_e len: 2
cos score:
tensor([[0.8548, 0.7613]])
eucl score:
[0.5388397, 0.6910004]
<class 'torch.Tensor'>
0.854826033115387
0.7612593173980713
label: False
outcome: False
=================================comparison end=========================================
===================================comparison start=======================================
original answer: Memory allocated with malloc must be expl

In [ ]:
a = "Memory allocated with malloc must be explicitly deallocated with delete"
b = "Memory allocated with malloc() must be explicitly deallocated with free()"

a = model.encode(a, convert_to_tensor=True)
b = model.encode(b, convert_to_tensor=True)

cosine_score = util.pytorch_cos_sim(a, b)
print("cos distance: " + str(cosine_score))
euc_score = np.linalg.norm(a.numpy() - b.numpy())
print("euc_score: " + str(euc_score))



cos distance: tensor([[0.8382]])
euc_score: 0.5688702
